In [323]:
import datetime
import pandas as pd
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression
import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot   as plt
from matplotlib import style
style.use('ggplot')
matplotlib.use('TkAgg')
import sqlite3

In [324]:
def query_db(db_name = 'QuanLyPhongGym.db', table_name ='DOANHTHU'):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    query = f"SELECT TIENTHU, NGAYTONGKETDOANHTHU, DOANHTHUGOITAP, DOANHTHUBANSANPHAM FROM {table_name}"
    cursor.execute(query)
    
    result = cursor.fetchall()
    conn.close()
    return result

In [325]:
data = query_db()
columns = ['TIENTHU', 'NGAYTONGKETDOANHTHU', 'DOANHTHUGOITAP', 'DOANHTHUBANSANPHAM']
df = pd.DataFrame(data, columns=columns)
print(df)

        TIENTHU NGAYTONGKETDOANHTHU  DOANHTHUGOITAP  DOANHTHUBANSANPHAM
0     750524295          2020-01-01       486457177           264067118
1     186833078          2020-01-02        44698650           142134428
2     854951836          2020-01-03       529744140           325207696
3     341047488          2020-01-04       148887348           192160140
4     280496762          2020-01-05        38357113           242139649
...         ...                 ...             ...                 ...
1642  503196357          2024-06-30       402140052           101056305
1643  737522220          2024-07-01       620364112           117158108
1644  855905929          2024-07-02       747909243           107996686
1645  842402115          2024-07-03       746483641            95918474
1646  555159585          2024-07-04       312042751           243116834

[1647 rows x 4 columns]


In [326]:
print(df.head())

     TIENTHU NGAYTONGKETDOANHTHU  DOANHTHUGOITAP  DOANHTHUBANSANPHAM
0  750524295          2020-01-01       486457177           264067118
1  186833078          2020-01-02        44698650           142134428
2  854951836          2020-01-03       529744140           325207696
3  341047488          2020-01-04       148887348           192160140
4  280496762          2020-01-05        38357113           242139649


In [327]:
import math
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import sqlite3

# Kết nối và lấy dữ liệu từ cơ sở dữ liệu
def query_db(db_name='QuanLyPhongGym.db', table_name='DOANHTHU'):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    query = f"SELECT TIENTHU, NGAYTONGKETDOANHTHU, DOANHTHUGOITAP, DOANHTHUBANSANPHAM FROM {table_name}"
    cursor.execute(query)
    
    result = cursor.fetchall()
    conn.close()
    return result

# Lấy dữ liệu từ cơ sở dữ liệu
df = pd.DataFrame(query_db(), columns=['TIENTHU', 'NGAYTONGKETDOANHTHU', 'DOANHTHUGOITAP', 'DOANHTHUBANSANPHAM'])

# Chuẩn bị dữ liệu
df = df[['TIENTHU', 'DOANHTHUGOITAP', 'DOANHTHUGOITAP']]
df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.01 * len(df)))
print(forecast_out)

df['label'] = df['TIENTHU'].shift(-forecast_out)
X = np.array(df.drop(['label'], axis=1))

# Sử dụng MinMaxScaler để chuẩn hóa dữ liệu
scaler = preprocessing.MinMaxScaler()
X = scaler.fit_transform(X)

# Loại bỏ các hàng cuối cùng từ X để phù hợp với nhãn y đã được dịch chuyển
X = X[:-forecast_out]
df.dropna(inplace=True)

# Tạo lại nhãn y
y = np.array(df['label'])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

# Xây dựng và huấn luyện mô hình hồi quy tuyến tính
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = clf.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

accuracy = clf.score(X_test, y_test)
print(f"Độ chính xác: {accuracy}")


17
Mean Squared Error: 7.081595385125047e+16
R^2 Score: 0.0030278723293079013
Độ chính xác: 0.0030278723293079013
